In [3]:
import re

In [4]:
import numpy as np
import pandas as pd

In [5]:
from tqdm import tqdm

In [6]:
import json

In [8]:
import matplotlib.pyplot as plt

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import random
from phylodm import PhyloDM
import dendropy

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [12]:
import seaborn as sns
from scipy import stats

In [14]:
from scipy.stats import entropy

### import embedding

In [13]:
co_embedding = pd.read_csv("../../data/social_niche_embedding_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
co_embedding = co_embedding.drop("<unk>")

embed_cos = cosine_similarity(co_embedding)
embed_cos = pd.DataFrame(data=embed_cos, index=co_embedding.index, columns=co_embedding.index)

phy_embedding = pd.read_csv("../../data/Embedding_list/PCA_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
phy_embedding = phy_embedding.loc[co_embedding.index]

embed_cos_phy = cosine_similarity(phy_embedding)
embed_cos_phy = pd.DataFrame(data=embed_cos_phy, index=phy_embedding.index, columns=phy_embedding.index)

In [18]:
tree = dendropy.Tree.get_from_path('../SNE_overview/Data/tree.tre', schema='newick')
pdm = PhyloDM.load_from_dendropy(tree)
dm = pdm.dm(norm=False)
labels = pdm.taxa()
dm = pd.DataFrame(data=dm, index=labels, columns=labels) 
dm = dm.loc[co_embedding.index.values, co_embedding.index.values]

### SNE with phylogeny

In [15]:
taxonomy = pd.read_csv("../Pretraining_data_profile/Data/taxmap_slv_ssu_ref_nr_138.2.txt", sep="\t", low_memory=False)

In [16]:
acc = []
for i in range(taxonomy.shape[0]):
    temp = taxonomy.iloc[i]
    acc.append(f"{temp[0]}.{temp[1]}.{temp[2]}")

taxonomy = taxonomy.loc[:, "path"].str.split(';', expand=True)
taxonomy.index = acc
taxonomy = taxonomy.iloc[:, 0: 7]
taxonomy = taxonomy.loc[co_embedding.index.values, ]
taxonomy.columns = ["k", "p", "c", "o", "f", "g", "s"]
taxonomy = taxonomy.loc[:, ["p", "c", "o", "f", "g"]]
remove_tax = ["Incertae Sedis", "Unknown Family", ]

In [19]:
names = ["Phylum", "Class", "Order", "Family", "Genus"]
phy_dist_mean = []
embed_cosine_mean = []
phy_dist_sd = []
embed_cosine_sd = []
keep_tax = []
tax = []
for j in range(0, 5):
    group = taxonomy.iloc[:, j].unique()
    group = group[[i not in remove_tax for i in group]]
    for i in group:
        
        fid_pick = taxonomy.loc[taxonomy.iloc[:, j] == i].index.values
        
        if len(fid_pick) > 1:
            
            keep_tax.append(i)
            tax.append(names[j])
            
            temp = embed_cos.loc[fid_pick, fid_pick]
            upper_triangle = np.triu(temp, k=1)
            upper_triangle = pd.DataFrame(data=upper_triangle)
            upper_triangle['id_1'] = upper_triangle.index.tolist()
            upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
            upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
            upper_triangle.columns = ["id_1", "id_2", "value"]
            embed_cosine_mean.append(np.mean(upper_triangle["value"].values))
            embed_cosine_sd.append(np.std(upper_triangle["value"].values))
            
            temp = dm.loc[fid_pick, fid_pick]
            upper_triangle = np.triu(temp, k=1)
            upper_triangle = pd.DataFrame(data=upper_triangle)
            upper_triangle['id_1'] = upper_triangle.index.tolist()
            upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
            upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
            upper_triangle.columns = ["id_1", "id_2", "value"]
            phy_dist_mean.append(np.mean(upper_triangle["value"].values))
            phy_dist_sd.append(np.std(upper_triangle["value"].values))
        

In [20]:
table_group = pd.DataFrame({"name": keep_tax, "phy_dist_mean":phy_dist_mean, "phy_dist_sd":phy_dist_sd, 
                            "embed_cosine_mean":embed_cosine_mean, "embed_cosine_sd":embed_cosine_sd,
                            "tax":tax})
table_group.to_csv("Data/tax_group.csv", index=None)

In [344]:
genome_id = np.unique(genome_pairs.id_1.tolist() + genome_pairs.id_2.tolist())
genome_id = pd.DataFrame({"id": genome_id})
genome_id.to_csv("Data/genome_id.txt", index=None, header=None)

In [345]:
genome_pairs.to_csv("/home/dongbiao/word_embedding_microbiome/HGT/HGT_network/genome_pairs_vsearch.txt", sep=" ", index=None, header=None)